# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [98]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [99]:
weatherCheck_cleandata = pd.read_csv('../WeatherPy/OutputFiles/CityData.csv')
weatherCheck_cleandata.head(10)

,city,lon,lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cayenne,-52.3333,4.9333,80.10,1.0,0.0,2.30,GF,1.659587e+09
1,provideniya,-173.3000,64.3833,42.91,100.0,75.0,6.71,RU,1.659587e+09
2,atuona,-139.0333,-9.8000,77.58,81.0,36.0,19.84,PF,1.659587e+09
3,bluff,168.3333,-46.6000,57.79,71.0,22.0,14.63,NZ,1.659587e+09
4,butaritari,172.7902,3.0707,82.22,73.0,22.0,14.99,KI,1.659587e+09
5,igdir,44.0450,39.9237,81.59,41.0,20.0,8.05,TR,1.659587e+09
6,zernograd,40.3083,46.8452,72.77,74.0,51.0,5.03,RU,1.659587e+09
7,mataura,168.8643,-46.1927,57.47,68.0,28.0,7.56,NZ,1.659587e+09
8,loei,101.5000,17.4167,78.17,65.0,100.0,4.09,TH,1.659587e+09
9,mar del plata,-57.5575,-38.0023,46.42,84.0,100.0,8.05,AR,1.659587e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [100]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = weatherCheck_cleandata[['lat', 'lon']].astype(float)
humidity = weatherCheck_cleandata['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '500px'}

# Add Heatmap layer to map:
fig = gmaps.figure(layout=layout)
heatmapLayer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius=2)
# Add layer
fig.add_layer(heatmapLayer)

# Display fig
fig 

Figure(layout=FigureLayout(height='500px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [101]:
#For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.

fitWeatherCriteria = weatherCheck_cleandata.loc[(weatherCheck_cleandata['Max Temp'] > 70) & (weatherCheck_cleandata['Max Temp'] <= 80) & 
                                  (weatherCheck_cleandata['Wind Speed'] <= 30) & (weatherCheck_cleandata['Humidity'] <= 30)]
fitWeatherCriteria
fitWeatherCriteria.reset_index(drop=True, inplace=True)
fitWeatherCriteria

,city,lon,lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,porto murtinho,-57.8825,-21.6989,76.46,30.0,92.0,5.01,BR,1.659587e+09
1,yulara,130.9889,-25.2406,72.16,27.0,0.0,20.98,AU,1.659587e+09
2,broken hill,141.4333,-31.9500,70.65,29.0,9.0,29.95,AU,1.659587e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [119]:

#resetting index to start from 0
fitWeatherCriteria.reset_index(drop=True, inplace=True)
fitWeatherCriteria
missingData=0

URL ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Add a "Hotel Name" column to the DataFrame.
fitWeatherCriteria["Hotel Name"]=""

# Set parameters to search for hotels with 5000 meters.
params={"radius": 5000,"types": "lodging","keyword": "hotel","key": g_key}

for index,row in fitWeatherCriteria.iterrows():
    
    lat = row['lat']
    lon = row['lon']
    #print(lat)
    #print(lon)
    params['location']=f"{lat},{lon}"
    
    #print("params",params)
    response=requests.get(URL, params=params).json()
    #print(response)
    
    results = response['results']
    #print(results)
    try: 
        # Store the first Hotel result into the DataFrame.
        fitWeatherCriteria.loc[index, 'Hotel Name'] = results[0]['name']
    except (IndexError):
        print('Result missing for a city')
        missingData+=1
print(missingData)


fitWeatherCriteria = fitWeatherCriteria[fitWeatherCriteria['Hotel Name'].notna()]
fitWeatherCriteria
# Plot markers on top of the heatmap.


0


,city,lon,lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,porto murtinho,-57.8825,-21.6989,76.46,30.0,92.0,5.01,BR,1.659587e+09,Hotel Pantanal
1,yulara,130.9889,-25.2406,72.16,27.0,0.0,20.98,AU,1.659587e+09,The Lost Camel Hotel - Ayers Rock Resort
2,broken hill,141.4333,-31.9500,70.65,29.0,9.0,29.95,AU,1.659587e+09,Palace Hotel


In [120]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fitWeatherCriteria.iterrows()]
locations = fitWeatherCriteria[["lat", "lon"]]

In [121]:
# Add marker layer ontop of heat map
markerLayer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markerLayer)
# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='1000px'))